In [1]:
import numpy as np
import sys

import librosa
from importlib import reload
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [2]:
# reload(librosa)
# librosa.onset.test()

In [3]:
audio_path = "/Applications/osu!.app/Contents/Resources/drive_c/osu!/Songs/13019 Daisuke Achiwa - BASARA/BASARA.mp3"

y, sr = librosa.load(audio_path,sr=None)

len(y), sr

(9107712, 44100)

# Precise Onset

In [4]:
for precise_window_size in [2,3,4]:
    print('precise_window_size: ', precise_window_size)
    for precise_hop_length in [4,8,16,32,64,128]:
        print('precise_hop_length: ', precise_hop_length)
        hop_length = 512
        onsets = librosa.onset.onset_detect(y=y, sr=sr, hop_length = hop_length, 
                                            precise=True, precise_hop_length=precise_hop_length,
                                            precise_window_size = precise_window_size,
                                            units='time')
        print('onsets: ', len(onsets))
        windowsize=1
        timeIntervals = [(w[windowsize]-w[0])/windowsize for w in window(onsets, windowsize+1)]
        quantiles = [40,45,50,55,60]
        print(60/np.percentile(timeIntervals, q=quantiles))

precise_window_size:  2
precise_hop_length:  4
onsets:  811
[ 271.55172414  261.21981559  258.90410959  258.29754002  257.79423227]
precise_hop_length:  8
onsets:  811
[ 271.55172414  261.22497334  258.90410959  258.3984375   257.79423227]
precise_hop_length:  16
onsets:  811
[ 271.55172414  261.23528947  258.8028169   258.3984375   257.59345794]
precise_hop_length:  32
onsets:  811
[ 271.10655738  260.84384858  259.20846395  258.3984375   257.59345794]
precise_hop_length:  64
onsets:  811
[ 271.99835526  260.02358491  258.3984375   258.3984375   258.3984375 ]
precise_hop_length:  128
onsets:  811
[ 271.99835526  261.6693038   258.3984375   258.3984375   258.3984375 ]
precise_window_size:  3
precise_hop_length:  4
onsets:  811
[ 265.4494382   262.80765181  260.79243051  259.61538462  258.25720309]
precise_hop_length:  8
onsets:  811
[ 265.4494382   262.70849881  260.84384858  259.61538462  258.31771321]
precise_hop_length:  16
onsets:  811
[ 265.4494382   262.5         260.84384858  25

In [5]:
librosa.frames_to_time(onsets, sr=sr)

array([  3.1230839 ,   3.36689342,   3.4829932 ,   3.58748299,
         3.69197279,   3.81968254,   3.93578231,   4.05188209,
         4.21442177,   4.28408163,   4.38857143,   4.5046712 ,
         4.73687075,   4.96907029,   5.21287982,   5.32897959,
         5.44507937,   5.56117914,   5.67727891,   5.79337868,
         5.89786848,   6.11845805,   6.23455782,   6.3506576 ,
         6.58285714,   6.69895692,   6.81505669,   7.04725624,
         7.17496599,   7.29106576,   7.51165533,   7.63936508,
         7.75546485,   7.99927438,   8.23147392,   8.46367347,
         8.57977324,   8.66104308,   8.92807256,   9.03256236,
         9.14866213,   9.2647619 ,   9.3692517 ,   9.61306122,
         9.729161  ,   9.82204082,   9.92653061,  10.05424036,
        10.29804989,  10.50702948,  10.97142857,  11.2152381 ,
        11.43582766,  11.66802721,  11.90022676,  12.1324263 ,
        12.36462585,  12.58521542,  12.71292517,  12.81741497,
        13.06122449,  13.28181406,  13.51401361,  13.74

In [6]:
samplesStarts = librosa.core.frames_to_samples(onsets, hop_length=hop_length)
samplesStarts

array([137728, 148480, 153600, 158208, 162816, 168448, 173568, 178688,
       185856, 188928, 193536, 198656, 208896, 219136, 229888, 235008,
       240128, 245248, 250368, 255488, 260096, 269824, 274944, 280064,
       290304, 295424, 300544, 310784, 316416, 321536, 331264, 336896,
       342016, 352768, 363008, 373248, 378368, 381952, 393728, 398336,
       403456, 408576, 413184, 423936, 429056, 433152, 437760, 443392,
       454144, 463360, 483840, 494592, 504320, 514560, 524800, 535040,
       545280, 555008, 560640, 565248, 576000, 585728, 595968, 606208,
       616448, 626176, 636928, 642048, 646656, 658432, 663040, 667136,
       677376, 682496, 687616, 698368, 708096, 728576, 758784, 769024,
       779264, 789504, 809984, 819712, 830464, 840192, 845312, 849920,
       860160, 870400, 881152])

In [7]:
onset_envelope = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
print("onset_envelope: ",len(onset_envelope))

onset_envelope:  1723


In [33]:
samplesStarts = librosa.core.frames_to_samples(onsets-2, hop_length=hop_length)
[print((end-start)//hop_length,np.argmax
    (librosa.onset.onset_strength(y[range(start, end)],
        sr=sr,hop_length=512))
)
for (start, end) in window(samplesStarts)]


21 3
10 3
9 3
9 3
11 9
10 3
10 3
14 3
6 3
9 3
10 3
20 3
20 3
21 3
10 3
10 3
10 10
10 8
10 3
9 3
19 3
10 3
10 3
20 3
10 3
10 3
20 3
11 3
10 3
19 3
11 3
10 5
21 3
20 3
20 3
10 3
7 3
23 3
9 8
10 10
10 3
9 3
21 3
10 3
8 3
9 3
11 10
21 3
18 3
40 3
21 3
19 4
20 3
20 3
20 3
20 12
19 5
11 3
9 3
21 3
19 4
20 3
20 4
20 3
19 12
21 3
10 3
9 9
23 3
9 8
8 3
20 3
10 4
10 3
21 12
19 10
40 3
59 42
20 13
20 3
20 13
40 22
19 3
21 13
19 9
10 3
9 3
20 3
20 3
21 3


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Algo tests

In [5]:
onsets = librosa.onset.onset_detect(y=y, sr=sr, hop_length = hop_length)
samplesStarts = librosa.core.frames_to_samples(onsets, hop_length=hop_length)
for startSample in samplesStarts:
    strength = librosa.onset.onset_strength(y[range(startSample-hop_length*3, startSample+hop_length*3)],sr=sr,hop_length=64)
    print(
        len(strength),
        np.argmax(strength),
        (np.argmax(strength)-(len(strength)-1)//2),
        (np.argmax(strength)-(len(strength)-1)//2)/(len(strength)-1)*6
    )

49 19 -5 -0.625
49 20 -4 -0.5
49 21 -3 -0.375
49 27 3 0.375
49 21 -3 -0.375
49 20 -4 -0.5
49 24 0 0.0
49 24 0 0.0
49 20 -4 -0.5
49 22 -2 -0.25
49 23 -1 -0.125
49 20 -4 -0.5
49 27 3 0.375
49 22 -2 -0.25
49 23 -1 -0.125
49 20 -4 -0.5
49 18 -6 -0.75
49 20 -4 -0.5
49 20 -4 -0.5
49 20 -4 -0.5
49 24 0 0.0
49 24 0 0.0
49 22 -2 -0.25
49 26 2 0.25
49 21 -3 -0.375
49 24 0 0.0
49 21 -3 -0.375
49 22 -2 -0.25
49 24 0 0.0
49 23 -1 -0.125
49 21 -3 -0.375
49 30 6 0.75
49 20 -4 -0.5
49 27 3 0.375
49 25 1 0.125
49 21 -3 -0.375
49 20 -4 -0.5
49 21 -3 -0.375
49 23 -1 -0.125
49 20 -4 -0.5
49 19 -5 -0.625
49 30 6 0.75
49 22 -2 -0.25
49 23 -1 -0.125
49 20 -4 -0.5
49 19 -5 -0.625
49 22 -2 -0.25
49 20 -4 -0.5
49 19 -5 -0.625
49 21 -3 -0.375
49 23 -1 -0.125
49 22 -2 -0.25
49 22 -2 -0.25
49 20 -4 -0.5
49 23 -1 -0.125
49 20 -4 -0.5
49 19 -5 -0.625
49 21 -3 -0.375
49 19 -5 -0.625
49 23 -1 -0.125
49 20 -4 -0.5
49 22 -2 -0.25
49 20 -4 -0.5
49 21 -3 -0.375
49 20 -4 -0.5
49 19 -5 -0.625
49 20 -4 -0.5
49 25 1 0.125
49 

# Dynamic BPM

In [1]:
import numpy as np
import sys

import librosa

#audio_path = "/Users/willian/Github/OSU-Mapper/osu-audio-feature-extract/Data/Beatmaps/355132 KOTOKO - BLAZE/fluff.mp3"
#audio_path = "/Users/willian/Github/OSU-Mapper/osu-audio-feature-extract/Data/Beatmaps/406217 Chata - enn/ehh.mp3"
audio_path = "/Applications/osu!.app/Contents/Resources/drive_c/osu!/Songs/13019 Daisuke Achiwa - BASARA/BASARA.mp3"
#audio_path = "/Applications/osu!.app/Contents/Resources/drive_c/osu!/Songs/151878 Chasers - Lost/Chasers - Lost.mp3"

y, sr = librosa.load(audio_path,sr=None)

len(y), sr

(9107712, 44100)

In [3]:
dynamic_tempo_summary = librosa.beat.dynamic_tempo_summary(y=y, sr=sr, precise=True, units='time', precise_show_original=True)
dynamic_tempo_summary

[(0.0, 0.39473922902494329, 246.09375, 246.09375),
 (0.39473922902494329,
  1.3815873015873017,
  126.04801829268293,
  126.04801829268293),
 (1.3815873015873017, 204.25433106575963, 129.19921875, 129.99263080324243),
 (204.25433106575963,
  204.98575963718821,
  132.51201923076923,
  132.51201923076923),
 (204.98575963718821,
  205.26439909297054,
  139.67483108108109,
  139.67483108108109),
 (205.26439909297054, 205.82167800453516, 143.5546875, 143.5546875),
 (205.82167800453516,
  206.51827664399093,
  156.60511363636363,
  156.60511363636363)]

In [4]:
np.array(dynamic_tempo_summary)

array([[   0.        ,    0.39473923,  246.09375   ,  246.09375   ],
       [   0.39473923,    1.3815873 ,  126.04801829,  126.04801829],
       [   1.3815873 ,  204.25433107,  129.19921875,  129.9926308 ],
       [ 204.25433107,  204.98575964,  132.51201923,  132.51201923],
       [ 204.98575964,  205.26439909,  139.67483108,  139.67483108],
       [ 205.26439909,  205.821678  ,  143.5546875 ,  143.5546875 ],
       [ 205.821678  ,  206.51827664,  156.60511364,  156.60511364]])

In [10]:
f = librosa.time_to_frames([0, 0.057], sr=sr, hop_length=512 )
f

array([0, 4])

In [15]:
t = librosa.frames_to_time([0, 1,4,4.5,5,6,7], sr=sr, hop_length=512 )
t

array([ 0.        ,  0.01160998,  0.04643991,  0.0522449 ,  0.05804989,
        0.06965986,  0.08126984])